##### Copyright 2023 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TF-NumPy Type Promotion

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/guide/tf_numpy_type_promotion"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/tf_numpy_type_promotion.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/guide/tf_numpy_type_promotion.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/guide/tf_numpy_type_promotion.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

## Overview

There are 4 options for type promotion in TensorFlow.

- By default, TensorFlow raises errors instead of promoting types for mixed type operations.
- Running `tf.numpy.experimental_enable_numpy_behavior()` switches TensorFlow to use [NumPy type promotion rules](https://www.tensorflow.org/guide/tf_numpy#type_promotion).
- **This doc** describes two new options that will be available in TensorFlow 2.15 (or currently in `tf-nightly`):

In [ ]:
!pip install -q tf_nightly

 **Note**: `experimental_enable_numpy_behavior` changes the behavior of all of TensorFlow.

## Setup

In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow.experimental.numpy as tnp

print("Using TensorFlow version %s" % tf.__version__)

### Enabling the new type promotion

In order to use the [JAX-like type promotion](https://jax.readthedocs.io/en/latest/type_promotion.html) in TF-Numpy, specify either `'all'` or `'safe'` as the dtype conversion mode when enabling NumPy behavior for TensorFlow.

This new system (with `dtype_conversion_mode="all"`) is associative, commutative, and makes it easy to control what width of float you end up with (it doesn't automatically convert to wider floats). It does introduce some risks of overflows and precision loss, but `dtype_conversion_mode="safe"` forces you to handle those cases explicitly. The two modes are explained more in detail in the [next section](#two_modes).

In [ ]:
tnp.experimental_enable_numpy_behavior(dtype_conversion_mode="all")

<a name="two_modes">

## Two Modes : ALL mode vs SAFE mode

In the new type promotion system, we introduce two modes: `ALL` mode and `SAFE` mode. `SAFE` mode is used to mitigate the concerns of "risky" promotions that can result in precision loss or bit-widening.

### Dtypes

We will be using the following abbreviations for brevity.

*   `b` means `tf.bool`
*   `u8` means `tf.uint8`
*   `i16` means `tf.int16`
*   `i32` means `tf.int32`
*   `bf16` means `tf.bfloat16`
*   `f32` means `tf.float32`
*   `f64` means `tf.float64`
*   `i32*` means Python `int` or weakly-typed `i32`
*   `f32*` means Python `float` or weakly-typed `f32`
*   `c128*` means Python `complex` or weakly-typed `c128`

The asterisk (*) denotes that the corresponding type is “weak” - such a dtype is temporarily inferred by the system, and could defer to other dtypes. This concept is explained more in detail [here](#weak_tensor).

### Example of precision losing operations

In the following example, `i32` + `f32` is allowed in `ALL` mode but
not in `SAFE` mode due to the risk of precision loss.

In [ ]:
# i32 + f32 returns a f32 result in ALL mode.
tnp.experimental_enable_numpy_behavior(dtype_conversion_mode="all")
a = tf.constant(10, dtype = tf.int32)
b = tf.constant(5.0, dtype = tf.float32)
a + b  # <tf.Tensor: shape=(), dtype=float32, numpy=15.0>

In [ ]:
# This promotion is not allowed in SAFE mode.
tnp.experimental_enable_numpy_behavior(dtype_conversion_mode="safe")
a = tf.constant(10, dtype = tf.int32)
b = tf.constant(5.0, dtype = tf.float32)
try:
  a + b
except TypeError as e:
   print(f'{type(e)}: {e}')  # TypeError: explicitly specify the dtype or switch to ALL mode.

### Example of bit-widening operations

In the following example, `i8` + `u32` is allowed in `ALL` mode but
not in `SAFE` mode due to bit-widening, which means using more bits than the number of bits in the inputs. Note that the new type promotion semantics only allows necessary bit-widening.

In [ ]:
# i8 + u32 returns an i64 result in ALL mode.
tnp.experimental_enable_numpy_behavior(dtype_conversion_mode="all")
a = tf.constant(10, dtype = tf.int8)
b = tf.constant(5, dtype = tf.uint32)
a + b

In [ ]:
# This promotion is not allowed in SAFE mode.
tnp.experimental_enable_numpy_behavior(dtype_conversion_mode="safe")
a = tf.constant(10, dtype = tf.int8)
b = tf.constant(5, dtype = tf.uint32)
try:
  a + b
except TypeError as e:
   print(f'{type(e)}: {e}')  # TypeError: explicitly specify the dtype or switch to ALL mode.

## A System Based on a Lattice

### Type Promotion Lattice

The new type promotion behavior is determined via the following type promotion lattice:

![Type Promotion Lattice](https://tensorflow.org/guide/images/new_type_promotion/type_promotion_lattice.png)

More specifically, promotion between any two types is determined by finding the first common child of the two nodes (including the nodes themselves).

For example, in the diagram above, the first common child of `i8` and `i32` is `i32` because the two nodes intersect for the first time at `i32` when following the direction of the arrows.

Similarly as another example, the result promotion type between `u64` and `f16` would be `f16`.

<a name="promotion_table">

### Type Promotion Table

Following the lattice generates the binary promotion table below:

**Note**: `SAFE` mode disallows the highlighted cells. `ALL` mode allows all cases.

![Type Promotion Table](https://tensorflow.org/guide/images/new_type_promotion/type_promotion_table.png)

## Advantages of The New Type Promotion

We adopt a JAX-like lattice-based system for our new type promotion, which offers the following advantages:

<a name="lattice_system_design">

#### Advantages of Lattice-Based System

First, using a lattice-based system ensures three very important properties:

*   Existence: There is a unique result promotion type for any combinations of types.
*   Commutativity: `a + b = b + a`
*   Associativity: `a + (b + c) = (a + b) = c`

These three properties are critical for constructing a type promotion semantics that is consistent and predictable.

#### Advantages of JAX-like Lattice System

Another crucial advantage of the JAX-like lattice system is that outside unsigned ints, it avoids all wider-than-necessary promotions. This means you cannot get 64-bit results without 64-bit inputs. This is especially beneficial for working on accelerators as it avoids unnecessary 64-bit values, which was frequent in the old type promotion.

However, this comes with a trade-off: mixed float/integer promotion is very prone to precision loss. For instance, in the example below, `i64` + `f16` results in promoting `i64` to `f16`.

In [ ]:
# The first input is promoted to f16 in ALL mode.
tnp.experimental_enable_numpy_behavior(dtype_conversion_mode="all")
tf.constant(1, tf.int64) + tf.constant(3.2, tf.float16)  # <tf.Tensor: shape=(), dtype=float16, numpy=4.2>

To migitage this concern, we introduced a `SAFE` mode that will disallow these "risky" promotions.

**Note**: To learn more about the design considerations in constructing the lattice system, please refer to the [Design of Type Promotion Semantics for JAX](https://jax.readthedocs.io/en/latest/jep/9407-type-promotion.html).

<a name="weak_tensor">

## WeakTensor

### Overview

*Weak tensors* are Tensors that are "weakly typed", similar to a [concept in JAX](https://jax.readthedocs.io/en/latest/type_promotion.html#weakly-typed-values-in-jax).

`WeakTensor`'s dtype is temporarily inferred by the system, and could defer to other dtypes. This concept is introduced in the new type promotion to prevent unwanted type promotion within binary operations between TF values and values with no explicitly user-specified type, such as Python scalar literals.

For instance, in the example below, `tf.constant(1.2)` is considered "weak" because it doesn't have a specific dtype. Therefore, `tf.constant(1.2)` defers to the type of `tf.constant(3.1, tf.float16)`, resulting in a `f16` output.

In [ ]:
tf.constant(1.2) + tf.constant(3.1, tf.float16)  # <tf.Tensor: shape=(), dtype=float16, numpy=4.3>

### WeakTensor Construction

WeakTensors are created if you create a tensor without specifying a dtype the result is a WeakTensor. You can check whether a Tensor is "weak" or not by checking the weak attribute at the end of the Tensor's string representation.

**First Case**: When `tf.constant` is called with an input with no user-specified dtype.

In [ ]:
tf.constant(5)  # <tf.Tensor: shape=(), dtype=int32, numpy=5, weak=True>

In [ ]:
tf.constant([5.0, 10.0, 3])  # <tf.Tensor: shape=(3,), dtype=float32, numpy=array([ 5., 10.,  3.], dtype=float32), weak=True>

In [ ]:
# A normal Tensor is created when dtype arg is specified.
tf.constant(5, tf.int32)  # <tf.Tensor: shape=(), dtype=int32, numpy=5>

**Second Case**: When an input with no user-specified dtype is passed into a [WeakTensor-supporting API](#weak_tensor_apis).

In [ ]:
tf.math.abs([100.0, 4.0])  # <tf.Tensor: shape=(2,), dtype=float32, numpy=array([100., 4.], dtype=float32), weak=True>

##Effects of turning on the new type promotion

Below is a non-exhaustive list of changes that result from turning on the new type promotion.

*   More consistent and predictable promotion results.
*   Reduced risk of bit-widening.
*   `tf.Tensor` mathematical dunder methods use new type promotion.
*   `tf.constant` can return `WeakTensor`.
*   `tf.constant` allows implicit conversions when a Tensor input with a dtype different from the `dtype` arg is passed in.
*   `tf.Variable` in-place ops (`assign`, `assign-add`, `assign-sub`) allow implicit conversions.
*   `tnp.array(1)` and `tnp.array(1.0)` returns 32-bit WeakTensor.
*   `WeakTensor`s will be created and used for [WeakTensor-supporting unary and binary API](#weak_tensor_apis)'s.


### More consistent and predictable promotion results

Using a [lattice-based system](#lattice_system_design) allows the new type promotion to produce consistent and predictable type promotion results.

#### Old Type Promotion

Changing the order of operations produces inconsistent results using old type promotion.

In [ ]:
# Setup
tnp.experimental_enable_numpy_behavior(dtype_conversion_mode="legacy")
a = np.array(1, dtype=np.int8)
b = tf.constant(1)
c = np.array(1, dtype=np.float16)

In [ ]:
# (a + b) + c throws an InvalidArgumentError.
try:
  tf.add(tf.add(a, b), c)
except tf.errors.InvalidArgumentError as e:
  print(f'{type(e)}: {e}')  # InvalidArgumentError

In [ ]:
# (b + a) + c returns an i32 result.
tf.add(tf.add(b, a), c)  # <tf.Tensor: shape=(), dtype=int32, numpy=3>

#### New Type Promotion

New type promotion produces consistent results regardless of the order.

In [ ]:
tnp.experimental_enable_numpy_behavior(dtype_conversion_mode="all")
a = np.array(1, dtype=np.int8)
b = tf.constant(1)
c = np.array(1, dtype=np.float16)

In [ ]:
# (a + b) + c returns a f16 result.
tf.add(tf.add(a, b), c)  # <tf.Tensor: shape=(), dtype=float16, numpy=3.0>

In [ ]:
# (b + a) + c also returns a f16 result.
tf.add(tf.add(b, a), c)  # <tf.Tensor: shape=(), dtype=float16, numpy=3.0>

### Reduced risk of bit-widening

#### Old Type Promotion

Old type promotion often resulted in 64-bit results.

In [ ]:
tnp.experimental_enable_numpy_behavior(dtype_conversion_mode="legacy")

In [ ]:
np.array(3.2, np.float16) + tf.constant(1, tf.int8) + tf.constant(50)  # <tf.Tensor: shape=(), dtype=float64, numpy=54.19921875>

#### New Type Promotion

New type promotion returns results with minimal number of bits necessary.

In [ ]:
tnp.experimental_enable_numpy_behavior(dtype_conversion_mode="all")

In [ ]:
np.array(3.2, np.float16) + tf.constant(1, tf.int8) + tf.constant(50)  # <tf.Tensor: shape=(), dtype=float16, numpy=54.2>

### tf.Tensor mathematical dunder methods

All `tf.Tensor` mathematical dunder methods will follow the new type promotion.

In [ ]:
-tf.constant(5)  # <tf.Tensor: shape=(), dtype=int32, numpy=-5, weak=True>

In [ ]:
tf.constant(5, tf.int16) - tf.constant(1, tf.float32)  # <tf.Tensor: shape=(), dtype=float32, numpy=4.0>

### tf.Variable in-place ops

Implicit conversions will be allowed in `tf.Variable` in-place ops.

**Note**: Any promotion that results in a dtype that is different from the variable's original dtype will be not allowed. This is because `tf.Variable` cannot change its dtype.

In [ ]:
tnp.experimental_enable_numpy_behavior(dtype_conversion_mode="all")
a = tf.Variable(10, tf.int32)
a.assign_add(tf.constant(5, tf.int16))  # <tf.Variable shape=() dtype=int32, numpy=15>

### tf.constant implicit conversions

In the old type promotion, `tf.constant` required an input Tensor to have the same dtype as the dtype argument. However, in the new type promotion, we implicitly convert Tensor to the specified dtype.

In [ ]:
tnp.experimental_enable_numpy_behavior(dtype_conversion_mode="all")
a = tf.constant(10, tf.int16)
tf.constant(a, tf.float32)  # <tf.Tensor: shape=(), dtype=float32, numpy=10.0>

### TF-NumPy Array

`tnp.array` defaults to `i32*` and `f32*` for python inputs using the new type promotion.

In [ ]:
tnp.array(1)  # <tf.Tensor: shape=(), dtype=int32, numpy=1, weak=True>

In [ ]:
tnp.array(1.0)  # <tf.Tensor: shape=(), dtype=int32, numpy=1, weak=True>

##Input Type Inference

This is how different inputs' types are inferred in the new type promotion.


*   `tf.Tensor`: Since `tf.Tensor` has a dtype property, we don't do further inference.
*   NumPy types: This includes types like `np.array(1)`, `np.int16(1)`, and `np.float`. Since NumPy inputs also have a dtype property, we take the dtype property as the result inference type. Note that NumPy defaults to `i64` and `f64`.
*   Python scalars/Nested types: This includes types like `1`, `[1, 2, 3]`, and `(1.0, 2.0)`.
   *   Python `int` is inferred as `i32*`.
   *   Python `float` is inferred as `f32*`.
   *   Python `complex` is inferred as `c128*`.
*  If the input doesn't fall into any of the above categories but has a dtype property, we take the dtype property as the result inference type.

# Further Reading

The new type promotion closely resembles JAX-NumPy's type promotion. If you want to know more details about the new type promotion and the design choices, check out the resources below.

*  [JAX Type Promotion Semantics](https://jax.readthedocs.io/en/latest/type_promotion.html)
*  [Design of Type Promotion Semantics for JAX](https://jax.readthedocs.io/en/latest/jep/9407-type-promotion.html)
*  [Old TF-NumPy Promotion Semantics](https://www.tensorflow.org/guide/tf_numpy#type_promotion)



# References

<a name="weak_tensor_apis">

## WeakTensor-supporting APIs

Below is a list of APIs that supports `WeakTensor`.

For an unary op, this means that if an input with no user-specified type is passed in, it will return a `WeakTensor`.

For a binary op, it will follow the promotion table [here](#promotion_table). It may or may not return a `WeakTensor` depending on the promotion result of the two inputs.

**Note**: All mathematical operations (`+`, `-`, `*`, ...) are supported.

* `tf.bitwise.invert`
* `tf.clip_by_value`
* `tf.debugging.check_numerics`
* `tf.expand_dims`
* `tf.identity`
* `tf.image.adjust_brightness`
* `tf.image.adjust_gamma`
* `tf.image.extract_patches`
* `tf.image.random_brightness`
* `tf.image.stateless_random_brightness`
* `tf.linalg.diag`
* `tf.linalg.diag_part`
* `tf.linalg.matmul`
* `tf.linalg.matrix_transpose`
* `tf.linalg.tensor_diag_part`
* `tf.linalg.trace`
* `tf.math.abs`
* `tf.math.acos`
* `tf.math.acosh`
* `tf.math.add`
* `tf.math.angle`
* `tf.math.asin`
* `tf.math.asinh`
* `tf.math.atan`
* `tf.math.atanh`
* `tf.math.ceil`
* `tf.math.conj`
* `tf.math.cos`
* `tf.math.cosh`
* `tf.math.digamma`
* `tf.math.divide_no_nan`
* `tf.math.divide`
* `tf.math.erf`
* `tf.math.erfc`
* `tf.math.erfcinv`
* `tf.math.erfinv`
* `tf.math.exp`
* `tf.math.expm1`
* `tf.math.floor`
* `tf.math.floordiv`
* `tf.math.floormod`
* `tf.math.imag`
* `tf.math.lgamma`
* `tf.math.log1p`
* `tf.math.log_sigmoid`
* `tf.math.log`
* `tf.math.multiply_no_nan`
* `tf.math.multiply`
* `tf.math.ndtri`
* `tf.math.negative`
* `tf.math.pow`
* `tf.math.real`
* `tf.math.real`
* `tf.math.reciprocal_no_nan`
* `tf.math.reciprocal`
* `tf.math.reduce_euclidean_norm`
* `tf.math.reduce_logsumexp`
* `tf.math.reduce_max`
* `tf.math.reduce_mean`
* `tf.math.reduce_min`
* `tf.math.reduce_prod`
* `tf.math.reduce_std`
* `tf.math.reduce_sum`
* `tf.math.reduce_variance`
* `tf.math.rint`
* `tf.math.round`
* `tf.math.rsqrt`
* `tf.math.scalar_mul`
* `tf.math.sigmoid`
* `tf.math.sign`
* `tf.math.sin`
* `tf.math.sinh`
* `tf.math.softplus`
* `tf.math.special.bessel_i0`
* `tf.math.special.bessel_i0e`
* `tf.math.special.bessel_i1`
* `tf.math.special.bessel_i1e`
* `tf.math.special.bessel_j0`
* `tf.math.special.bessel_j1`
* `tf.math.special.bessel_k0`
* `tf.math.special.bessel_k0e`
* `tf.math.special.bessel_k1`
* `tf.math.special.bessel_k1e`
* `tf.math.special.bessel_y0`
* `tf.math.special.bessel_y1`
* `tf.math.special.dawsn`
* `tf.math.special.expint`
* `tf.math.special.fresnel_cos`
* `tf.math.special.fresnel_sin`
* `tf.math.special.spence`
* `tf.math.sqrt`
* `tf.math.square`
* `tf.math.subtract`
* `tf.math.tan`
* `tf.math.tanh`
* `tf.nn.depth_to_space`
* `tf.nn.elu`
* `tf.nn.gelu`
* `tf.nn.leaky_relu`
* `tf.nn.log_softmax`
* `tf.nn.relu6`
* `tf.nn.relu`
* `tf.nn.selu`
* `tf.nn.softsign`
* `tf.nn.space_to_depth`
* `tf.nn.swish`
* `tf.ones_like`
* `tf.realdiv`
* `tf.reshape`
* `tf.squeeze`
* `tf.stop_gradient`
* `tf.transpose`
* `tf.truncatediv`
* `tf.truncatemod`
* `tf.zeros_like`
* `tf.experimental.numpy.abs`
* `tf.experimental.numpy.absolute`
* `tf.experimental.numpy.amax`
* `tf.experimental.numpy.amin`
* `tf.experimental.numpy.angle`
* `tf.experimental.numpy.arange`
* `tf.experimental.numpy.arccos`
* `tf.experimental.numpy.arccosh`
* `tf.experimental.numpy.arcsin`
* `tf.experimental.numpy.arcsinh`
* `tf.experimental.numpy.arctan`
* `tf.experimental.numpy.arctanh`
* `tf.experimental.numpy.around`
* `tf.experimental.numpy.array`
* `tf.experimental.numpy.asanyarray`
* `tf.experimental.numpy.asarray`
* `tf.experimental.numpy.ascontiguousarray`
* `tf.experimental.numpy.average`
* `tf.experimental.numpy.bitwise_not`
* `tf.experimental.numpy.cbrt`
* `tf.experimental.numpy.ceil`
* `tf.experimental.numpy.conj`
* `tf.experimental.numpy.conjugate`
* `tf.experimental.numpy.copy`
* `tf.experimental.numpy.cos`
* `tf.experimental.numpy.cosh`
* `tf.experimental.numpy.cumprod`
* `tf.experimental.numpy.cumsum`
* `tf.experimental.numpy.deg2rad`
* `tf.experimental.numpy.diag`
* `tf.experimental.numpy.diagflat`
* `tf.experimental.numpy.diagonal`
* `tf.experimental.numpy.diff`
* `tf.experimental.numpy.empty_like`
* `tf.experimental.numpy.exp2`
* `tf.experimental.numpy.exp`
* `tf.experimental.numpy.expand_dims`
* `tf.experimental.numpy.expm1`
* `tf.experimental.numpy.fabs`
* `tf.experimental.numpy.fix`
* `tf.experimental.numpy.flatten`
* `tf.experimental.numpy.flip`
* `tf.experimental.numpy.fliplr`
* `tf.experimental.numpy.flipud`
* `tf.experimental.numpy.floor`
* `tf.experimental.numpy.full_like`
* `tf.experimental.numpy.imag`
* `tf.experimental.numpy.log10`
* `tf.experimental.numpy.log1p`
* `tf.experimental.numpy.log2`
* `tf.experimental.numpy.log`
* `tf.experimental.numpy.max`
* `tf.experimental.numpy.mean`
* `tf.experimental.numpy.min`
* `tf.experimental.numpy.moveaxis`
* `tf.experimental.numpy.nanmean`
* `tf.experimental.numpy.negative`
* `tf.experimental.numpy.ones_like`
* `tf.experimental.numpy.positive`
* `tf.experimental.numpy.prod`
* `tf.experimental.numpy.rad2deg`
* `tf.experimental.numpy.ravel`
* `tf.experimental.numpy.real`
* `tf.experimental.numpy.reciprocal`
* `tf.experimental.numpy.repeat`
* `tf.experimental.numpy.reshape`
* `tf.experimental.numpy.rot90`
* `tf.experimental.numpy.round`
* `tf.experimental.numpy.signbit`
* `tf.experimental.numpy.sin`
* `tf.experimental.numpy.sinc`
* `tf.experimental.numpy.sinh`
* `tf.experimental.numpy.sort`
* `tf.experimental.numpy.sqrt`
* `tf.experimental.numpy.square`
* `tf.experimental.numpy.squeeze`
* `tf.experimental.numpy.std`
* `tf.experimental.numpy.sum`
* `tf.experimental.numpy.swapaxes`
* `tf.experimental.numpy.tan`
* `tf.experimental.numpy.tanh`
* `tf.experimental.numpy.trace`
* `tf.experimental.numpy.transpose`
* `tf.experimental.numpy.triu`
* `tf.experimental.numpy.vander`
* `tf.experimental.numpy.var`
* `tf.experimental.numpy.zeros_like`